In [1]:
import pandas as pd
import numpy as np

In [13]:
pbps = pd.read_csv('./res/pbps.csv')[['year', 'home_team', 'home_team_id', 'away_team', 'away_team_id', 'bat_name', 'bat_id',
                                      'pitcher_name', 'pitcher_id', 'game_state', 'next_game_state', 'WPA', 'LI']]

# Add 0.1 to zero LI to avoid division by zero
pbps['LI'] = pbps['LI'].apply(lambda x: x + 0.1 if x == 0 else x)
pbps['WPA/LI'] = pbps['WPA'] / pbps['LI']

def aggregate(group):
    return pd.Series({
        'n_plays': len(group),
        't_WPA/LI': np.nansum(group['WPA/LI']),
        'WPA': np.nansum(group['WPA']),
        'pLI': np.nanmean(group['LI']),
    })

In [27]:
# top_pbps = pbps.loc[pbps['top_inning'] == 1]
# bot_pbps = pbps.loc[pbps['top_inning'] == 0]

# top_batters = top_pbps.groupby(['away_team', 'bat_name'])
# top_pitchers = top_pbps.groupby(['home_team', 'pitcher_name'])

# bot_batters = bot_pbps.groupby(['home_team', 'bat_name'])
# bot_pitchers = bot_pbps.groupby(['away_team', 'pitcher_name'])

# top_batters = top_batters.apply(aggregate).reset_index().rename(columns={'away_team': 'team', 'bat_name': 'name'})
# top_batters['WPA/pLI'] = top_batters['WPA'] / top_batters['pLI']
# top_batters['Clutch'] = top_batters['WPA/pLI'] - top_batters['t_WPA/LI']

# top_pitchers = top_pitchers.apply(aggregate).reset_index().rename(columns={'home_team': 'team', 'pitcher_name': 'name'})
# top_pitchers['WPA/pLI'] = top_pitchers['WPA'] / top_pitchers['pLI']
# top_pitchers['Clutch'] = top_pitchers['WPA/pLI'] - top_pitchers['t_WPA/LI']

# bot_batters = bot_batters.apply(aggregate).reset_index().rename(columns={'home_team': 'team', 'bat_name': 'name'})
# bot_batters['WPA/pLI'] = bot_batters['WPA'] / bot_batters['pLI']
# bot_batters['Clutch'] = bot_batters['WPA/pLI'] - bot_batters['t_WPA/LI']

# bot_pitchers = bot_pitchers.apply(aggregate).reset_index().rename(columns={'away_team': 'team', 'pitcher_name': 'name'})
# bot_pitchers['WPA/pLI'] = bot_pitchers['WPA'] / bot_pitchers['pLI']
# bot_pitchers['Clutch'] = bot_pitchers['WPA/pLI'] - bot_pitchers['t_WPA/LI']

# batters = pd.concat([top_batters, bot_batters])
# pitchers = pd.concat([top_pitchers, bot_pitchers])

In [14]:
batters = pbps.groupby(['bat_id', 'bat_name']).apply(aggregate).reset_index()
batters['WPA/pLI'] = batters['WPA'] / batters['pLI']
batters['Clutch'] = batters['WPA/pLI'] - batters['t_WPA/LI']

pitchers = pbps.groupby(['pitcher_id', 'pitcher_name']).apply(aggregate).reset_index()
pitchers['WPA/pLI'] = pitchers['WPA'] / pitchers['pLI']
pitchers['Clutch'] = pitchers['WPA/pLI'] - pitchers['t_WPA/LI']

In [15]:
batters.sort_values('Clutch', ascending=False, inplace=True)
pitchers.sort_values('Clutch', ascending=False, inplace=True)

batters.to_csv('./res/batters.csv', index=False)

batters.head()

,bat_id,bat_name,n_plays,t_WPA/LI,WPA,pLI,WPA/pLI,Clutch
16263,2826297.0,PARKER,83.0,-123.256862,0.288028,1.019385,0.282550,123.539413
13888,2805749.0,"Adams, Billy",72.0,-122.974946,0.335979,1.026105,0.327431,123.302377
14981,2812669.0,Caskey,108.0,-122.735312,0.387089,0.971424,0.398476,123.133788
12975,2797428.0,"KAMAKA, T.",134.0,-122.400094,0.479385,1.032674,0.464217,122.864311
15672,2822199.0,J. Martinez,29.0,-123.364811,-0.392700,0.607503,-0.646417,122.718394
